In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oliverli0414","key":"19df8e5218b8495a8c1ea362134c4a27"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!unzip home-credit-default-risk.zip

100% 688M/688M [00:04<00:00, 138MB/s]
100% 688M/688M [00:04<00:00, 145MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [4]:
import pandas as pd
import numpy as np

In [5]:
application_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
installments_payments = pd.read_csv('installments_payments.csv')

In [6]:
print(installments_payments.isna().sum())

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64


In [7]:
installments_payments['DIFF_DAY_PAYMENT'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
installments_payments['DIFF_AMT_PAYMENT'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']
installments_payments_sorted = pd.DataFrame()
installments_payments_sorted = installments_payments.groupby('SK_ID_PREV').max('MONTHS_BALANCE')[['SK_ID_CURR']]
installments_payments_sorted['OCCURENCES']=installments_payments.groupby('SK_ID_PREV').count()['NUM_INSTALMENT_VERSION']
installments_payments_sorted['DISTINCT_NUM_INSTALMENT_VERSION'] = installments_payments.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].nunique()
installments_payments_sorted['SUM_DIFF_DAY_PAYMENT'] = installments_payments.groupby('SK_ID_PREV')['DIFF_DAY_PAYMENT'].sum()
installments_payments_sorted['AVG_DIFF_DAY_PAYMENT'] = installments_payments_sorted['SUM_DIFF_DAY_PAYMENT']/installments_payments_sorted['OCCURENCES']
installments_payments_sorted['SUM_DIFF_AMT_PAYMENT'] = installments_payments.groupby('SK_ID_PREV')['DIFF_AMT_PAYMENT'].sum()
installments_payments_sorted['AVG_DIFF_AMT_PAYMENT'] = installments_payments_sorted['SUM_DIFF_AMT_PAYMENT']/installments_payments_sorted['OCCURENCES']
installments_payments_ID_CURR = installments_payments_sorted.groupby('SK_ID_CURR').sum()

In [8]:
installments_payments_ID_CURR__application_train = installments_payments_ID_CURR.merge(application_train, on='SK_ID_CURR', how='inner')
print(installments_payments_ID_CURR__application_train)

        SK_ID_CURR  OCCURENCES  DISTINCT_NUM_INSTALMENT_VERSION  \
0           100002          19                                2   
1           100003          25                                4   
2           100004           3                                2   
3           100006          16                                4   
4           100007          66                                6   
...            ...         ...                              ...   
291638      456251           7                                2   
291639      456252           6                                1   
291640      456253          14                                3   
291641      456254          19                                2   
291642      456255          74                               14   

        SUM_DIFF_DAY_PAYMENT  AVG_DIFF_DAY_PAYMENT  SUM_DIFF_AMT_PAYMENT  \
0                     -388.0            -20.421053                 0.000   
1                     -179.0            -22

In [9]:
corr = installments_payments_ID_CURR__application_train.corr(method='spearman')
print(corr)

                                 SK_ID_CURR  OCCURENCES  \
SK_ID_CURR                         1.000000    0.004610   
OCCURENCES                         0.004610    1.000000   
DISTINCT_NUM_INSTALMENT_VERSION    0.002459    0.696216   
SUM_DIFF_DAY_PAYMENT              -0.002597   -0.700930   
AVG_DIFF_DAY_PAYMENT              -0.002148   -0.428265   
SUM_DIFF_AMT_PAYMENT               0.003322    0.316405   
AVG_DIFF_AMT_PAYMENT               0.003094    0.285962   
TARGET                            -0.002363   -0.022472   

                                 DISTINCT_NUM_INSTALMENT_VERSION  \
SK_ID_CURR                                              0.002459   
OCCURENCES                                              0.696216   
DISTINCT_NUM_INSTALMENT_VERSION                         1.000000   
SUM_DIFF_DAY_PAYMENT                                   -0.631168   
AVG_DIFF_DAY_PAYMENT                                   -0.672390   
SUM_DIFF_AMT_PAYMENT                                    0.02

In [10]:
corr.to_excel(r'installments_payments.spearman.xlsx',index=True,header=True)

Choose AVG_DIFF_DAY_PAYMENT and AVG_DIFF_AMT_PAYMENT due to correlation.
Others with high correlation are already related.